# Analysis - Experiment 5: IT Split Range

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pylab as plt
import scipy.stats as ss

def swap_ioi_bpm(t):
    """
    Converts an interonset interval (IOI) in milliseconds to tempo in beats per minute, or
    BPM to the corresponding IOI. Conveniently, the equation is the same to convert in either
    direction - just divide 60000 ms by your value. Sometimes the universe is benign. :)
    :param t: Either an interonset interval in milliseconds or a BPM value. Can also be an array
        of these values.
    :return: If t was an interonset interval, result will be the corresponding BPM.
        If t was a tempo in BPM, result will be the corresponding interonset interval.
    """
    return 60000 / t

# Set file paths
DATAFILE = '../data/response_data.csv'
SURVEYFILE = '../data/survey_responses.csv'
EXCFILE = '../data/excluded.txt'
FIGURE_PATH = './figures/'
IOI_LEVELS = np.array([1000, 918, 843, 774, 710, 652, 599, 550, 504, 463, 425, 390, 358, 329, 302])
IOI_BINS = [(IOI_LEVELS[3*i], IOI_LEVELS[1+3*i], IOI_LEVELS[2+3*i]) for i in range(5)]
TEMPO_LEVELS = swap_ioi_bpm(IOI_LEVELS)
TEMPO_BINS = [(TEMPO_LEVELS[3*i], TEMPO_LEVELS[1+3*i], TEMPO_LEVELS[2+3*i]) for i in range(5)]
PITCH_LEVELS = [2, 3, 4, 5, 6, 7]
LOUDNESS_LEVELS = [0, 1, 2]
METRONOME_IOI = 550
METRONOME_TEMPO = swap_ioi_bpm(550)

# Define functions to convert between tempos and ratings
def bpm_to_rating(bpm, referent=METRONOME_TEMPO, intercept=50, slope=50):
    """
    Calculates location of any tempo in BPM on the scale used in the study. Appears
    in the manuscript as Equation 1.

    The default intercept and slope are the ground truth values, and assume 1) that a
    score of 50 corresponds to a tempo equal to the metronome and 2) every doubling of
    the tempo increases the score by 50. Subject-specific slopes and intercepts can be
    passed as arguments instead to obtain r_hat (see Equation 3).
    """
    return intercept + slope * np.log2(bpm / referent)

def rating_to_bpm(r, referent=METRONOME_TEMPO, intercept=50, slope=50):
    """
    Converts any relative tempo rating to its corresponding tempo in BPM.
    Appears in the manuscript as Equation 2.

    The default intercept and slope used in the equation are the ground truth values,
    but subject-specific slopes and intercepts can be passed as arguments instead to
    obtain t_hat (see Equation 4).
    """
    return referent * 2 ** ((r - intercept) / slope)

# Adjust matplotlib settings
plt.rc('figure', titlesize=32)  # fontsize of the figure title
plt.rc('axes', labelsize=28)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=23)    # fontsize of the tick labels
plt.rc('ytick', labelsize=23)    # fontsize of the tick labels
plt.rc('legend', fontsize=18)    # legend fontsize
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['axes.spines.top'] = False

In [ ]:
# Load data
data = pd.read_csv(DATAFILE)

# Remove excluded participants
excluded = np.loadtxt(EXCFILE, dtype=int)
data = data[~np.isin(data.subject, excluded)]

# Exclude outlier trials
data = data[data.cooks <= 4 / 180]

# Raw Tempo Ratings

In [ ]:
# Get subject averages
subj_avg = data.groupby(['subject', 'tempo']).response.mean().reset_index()
subj_slopes = data.groupby('subject').slope.mean().reset_index()
subj_intercepts = data.groupby('subject').intercept.mean().reset_index()

plt.figure()

# Scatterplot
ax = sns.scatterplot(x=np.log2(subj_avg.tempo), y='response', data=subj_avg, alpha=.075, color='k', s=75, zorder=1)

# Individual subject fits
for i in range(len(subj_slopes)):
    slope = subj_slopes.iloc[i, 1]
    intercept = subj_intercepts.iloc[i, 1]
    plt.axline((np.log2(TEMPO_LEVELS[0]), bpm_to_rating(TEMPO_LEVELS[0], intercept=intercept, slope=slope)),
               (np.log2(TEMPO_LEVELS[-1]), bpm_to_rating(TEMPO_LEVELS[-1], intercept=intercept, slope=slope)),
               c='#5E6A71', ls='-', alpha=.12, zorder=0, lw=.8, label='Individual fit' if i==0 else None)
        
# Average of all subject fits
inter = subj_intercepts.intercept.mean()
slope = subj_slopes.slope.mean()
plt.axline((np.log2(TEMPO_LEVELS[0]), bpm_to_rating(TEMPO_LEVELS[0], intercept=inter, slope=slope)),
           (np.log2(TEMPO_LEVELS[-1]), bpm_to_rating(TEMPO_LEVELS[-1], intercept=inter, slope=slope)),
           c='#019AFF', ls='-', alpha=1, label='Average fit', lw=4.5, zorder=2)

# Ground truth line
plt.axline((np.log2(TEMPO_LEVELS[0]), bpm_to_rating(TEMPO_LEVELS[0], intercept=50, slope=50)),
           (np.log2(TEMPO_LEVELS[-1]), bpm_to_rating(TEMPO_LEVELS[-1], intercept=50, slope=50)),
           c='k', ls='--', alpha=.8, label='Ground truth', lw=4.5, zorder=2)

plt.xlabel('Stimulus Tempo (BPM)')
plt.ylabel('Tempo Rating')
plt.yticks(range(0, 101, 25))
plt.xticks(np.log2([65, 85, 110, 140, 180]))
ax.set_xticklabels([65, 85, 110, 140, 180], rotation=0)
plt.xlim(np.log2(swap_ioi_bpm(1100)), np.log2(swap_ioi_bpm(275)))
plt.ylim(-.8, 100.8)
plt.legend()

plt.gcf().set_size_inches(8, 6.5)
plt.tight_layout()

plt.gcf().savefig(FIGURE_PATH + 'raw_ratings5.svg')

# Effect of Pitch and Tone Type

In [ ]:
plt.figure()
sns.lineplot(x=data.pitch, y='illusory_tempo', data=data[data.range==0], color='#002c6a', ls='-',
             marker='', err_style='bars', err_kws=dict(capsize=6, lw=3, capthick=3), lw=3, label='Lower Register')
sns.lineplot(x=data.pitch+5, y='illusory_tempo', data=data[data.range==1], color='#e45171', ls='--',
             marker='', err_style='bars', err_kws=dict(capsize=6, lw=3, capthick=3), lw=3, label='Upper Register')
plt.axhline(0, ls='--', c='k', alpha=.5)
plt.xticks(range(0, 11, 2), ['A2\n(110)', 'A3\n(220)', 'A4\n(440)', 'A5\n(880)', 'A6\n(1760)', 'A7\n(3560)'])
plt.yticks(range(-2, 6))
plt.xlabel('Pitch (Hz)')
plt.ylabel('Illusory Tempo (%)')
plt.legend(loc=2)
plt.gcf().set_size_inches(9, 7.5)
plt.tight_layout()
#plt.gcf().savefig(FIGURE_PATH + 'illusory_pitch5.svg')

In [ ]:
plt.figure()
sns.lineplot(x=data.pitch, y='residual', data=data[data.range==0], color='#002c6a', ls='-',
             marker='', err_style='bars', err_kws=dict(capsize=6, lw=3, capthick=3), lw=3, label='Lower Register')
sns.lineplot(x=data.pitch+5, y='residual', data=data[data.range==1], color='#e45171', ls='--',
             marker='', err_style='bars', err_kws=dict(capsize=6, lw=3, capthick=3), lw=3, label='Upper Register')
plt.axhline(0, ls='--', c='k', alpha=.5)
plt.ylim(-3, 3)
plt.xticks(range(0, 11, 2), ['A2', 'A3', 'A4', 'A5', 'A6', 'A7'])
plt.xlabel('Pitch')
plt.ylabel('Residual Tempo Rating')
plt.legend(loc=2)
plt.gcf().set_size_inches(9, 7.5)
plt.tight_layout()
plt.gcf().savefig(FIGURE_PATH + 'pitch5.svg')

In [ ]:
plt.figure()
sns.lineplot(x=data.pitch-.025, y='response', data=data[data.range==0], color='#002c6a', 
             marker='', err_style='bars', err_kws=dict(capsize=6, lw=3, capthick=3), lw=2.5, label='Low Register')
ax=sns.lineplot(x=data.pitch+5.025, y='response', data=data[data.range==1], color='#e45171', 
             marker='', err_style='bars', err_kws=dict(capsize=6, lw=3, capthick=3), lw=2.5, label='High Register')
plt.xlabel('Pitch Level')
plt.ylabel('Tempo Rating')
plt.ylim(46, 56.5)
plt.xticks([0, 2, 4, 6, 8, 10], ['A2', 'A3', 'A4', 'A5', 'A6', 'A7'])
plt.xlabel('Pitch')

plt.gcf().set_size_inches(12, 7.5)
plt.tight_layout()
plt.gcf().savefig(FIGURE_PATH + 'raw_pitch5.svg')

# Survey Analysis

Please note that in order to preserve participant privacy, survey responses have not been made open access. Please contact the authors for access to data regarding demographics and musical experience.

In [ ]:
# Load survey data and rename ID to subject
surv = pd.read_csv(SURVEYFILE)
surv.rename(columns={'id':'subject'}, inplace=True)

# Get list of subjects included in analyses
subj_list = data.subject.unique()

# Select only survey responses from participants included in analyses
surv = surv[np.in1d(surv.subject, subj_list)]
surv.reset_index()

# Add column indicating whether the participant was in the low or high range condition
surv['range'] = np.array([data[data.subject == s].range.iloc[0] for s in surv.subject])

# Print age stats
print('Minimum Age:', surv.age.min())
print('Maximum Age:', surv.age.max())
print('Mean Age:', surv.age.mean())
print('StdDev Age:', surv.age.std())

# Print gender stats
print('Number of Males:', np.sum(surv.gender == 'Male'))
print('Number of Females:', np.sum(surv.gender == 'Female'))
print('Number not reported:', np.sum(surv.gender== 'Prefer not to answer'))
print('Number self-described:', np.sum(surv.gender == 'Other'))
print('Self-descriptions:', [g for g in surv['gender[other]'] if type(g) == str])
print('Number of Low Females:', np.sum((surv.gender == 'Female') & (surv.range == 0)))
print('Number of Low Males:', np.sum((surv.gender == 'Male') & (surv.range == 0)))
print('Number of High Females:', np.sum((surv.gender == 'Female') & (surv.range == 1)))
print('Number of High Males:', np.sum((surv.gender == 'Male') & (surv.range == 1)))